# Quiz 2
# Machine Learning 2021-1

A logistic regression model is a statistical classification method that uses a generalized linear regression model to estimate $P(C=1 | \mathbf{x})$, the probability of the sample $\mathbf{x}\in\mathbb{R}^2$ belonging to class $C_1$. 

\begin{equation}
y=P(C=1|\mathbf{x},\mathbf{w})=\sigma(w_0+w_1x_0 + w_2x_1)
\end{equation}
where 
\begin{equation}
\sigma(x)=\frac{1}{1+e^{-x}}
\end{equation}

The following function implements a logistic regression classifier:

In [9]:
import numpy as np

def log_reg(w, x):
    ''' 
     w: weight vector with shape (3,)
     x: input sample with shape (2,)
     returns: P(C=1|x,w)
    '''
    return  1 / (1 +  np.exp(-(np.matmul(x, w[1:]) + w[0])))

### 1. Implement function *f_1*:
Assume a cost matrix $L$ where $L_{i,j}$ is the cost of predicting class $C_j$ when the real class is $C_i$. Write a function that calculates the risk of classifying a sample $\mathbf{x}$ in class $y \in \{0,1\}$.

**Weight of point is 2.**

In [11]:
def f_1(w, L, x, y):
    ''' 
     w: weight vector with shape (3,)
     L: loss matrix with shape (2,2)
     x: input sample with shape (2,)
     y: class value {0, 1}
     returns: R(y|x,w)
    '''
    #El riesgo se calcularia como R(Ci|x) = sum over k(lambda_{ik}*P(Ck|x)), tenemos la clase 0 y 1
    #Si i=k entonces el riesgo es 0
    #Si i=0 y k=1 (Riesgo de clasificar en 0 cuando es 1), entonces R(C0|x,w) = lambda_{[1][0]}*P(C1|x)
    #Si i=1 y k=0 (Riesgo de clasificar en 1 cuando es 0), entonces R(C1|x,w) = lambda_{[0][1]}*P(C0|x)
    #La probabilidad P(C1|x,w)=log_reg(w,x) y la probabilidad de P(C0|x,w)=1-P(C1|x,w)=1-log_reg(w,x), juntando esto:
    return L[0][y]*(1-log_reg(w,x)) + L[1][y]*(log_reg(w,x))

In [12]:
# TEST CELL
lst = [[x1, x2] for x1 in np.linspace(-2, 1.5, 4) for x2 in np.linspace(-2, 1.5, 4)]
X = np.array(lst)
W1 = np.array([0, 1, 2])
L1 = np.array([[0.01, 2],[0.9, -0.05]]) 

EPS = 0.0001
################## TEST 1 ##################
# Name: f_1 function
# Weight: 2
# Setup code:
R10 = np.array([ 0.01220063,  0.03218274,  0.19566159,  0.66064213,  0.0170284 , 0.07751378,  0.41800227,  0.80852222,  0.03218274,  0.19566159, 0.66064213,  0.86934378,  0.07751378,  0.41800227,  0.80852222, 0.89022162])
R11 = np.array([ 1.99493112,  1.94890493,  1.57235252,  0.50132991,  1.98381098, 1.84449073,  1.06021949,  0.16070725,  1.94890493,  1.57235252, 0.50132991,  0.02061265,  1.84449073,  1.06021949,  0.16070725, -0.02747677])
# TEST CASES:
# CASE 1: code
evaluate = lambda i: ((abs(R10[i] - f_1(W1, L1, X[i, :], 0)) < EPS) and (abs(R11[i] - f_1(W1, L1, X[i, :], 1)) < EPS) )
results = [evaluate(i) for i in range(len(lst))]
results == [True] * len(lst)

True

### 2. Implement function f_2:
Write a function that implements a classifier that makes the prediction that minimizes the risk.

**Weight of point is 1.5**.


In [14]:
def f_2(w, L, x):
    ''' 
     w: weight vector with shape (3,)
     L: loss vector with shape (2, 2)
     x: input sample with shape (2,)
     returns: predicted class {0, 1} 
    '''
    #Ya que debe minimizarse el riesgo, el clasificador elegira 1 si el riesgo de clasificar 
    #en 1 es menor que en clasificar en 0 y viceversa, entonces
    return 1 if f_1(w,L,x,1) < f_1(w,L,x,0) else 0

In [15]:
# TEST CELL
lst = [[x1, x2] for x1 in np.linspace(-2, 1.5, 4) for x2 in np.linspace(-2, 1.5, 4)]
X = np.array(lst)
W1 = np.array([0, 1, 2])
L1 = np.array([[0.01, 2],[0.9, -0.05]]) 

EPS = 0.0001

C1= np.array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0., 0.,  1.,  1.])
# TEST CASES:
# CASE 1: code
evaluate = lambda i: ((abs(C1[i] - f_2(W1, L1, X[i, :])) < EPS))
results = [evaluate(i) for i in range(len(lst))]
results == [True] * len(lst)

True

### 3. Implement function f_3:
Write a function that implements a classifier that makes the prediction that minimizes the risk, but that can also reject the sample. The cost of rejection is $\lambda$.

**Weight of point is 1.5.**

In [18]:
def f_3(w, L, lamb, x):
    ''' 
     w: weight vector with shape (3,)
     L: loss vector with shape (3,)
     x: input sample with shape (2,1)
     lamb: a float, the cost of rejection
     returns: predicted class {0, 1, 2}. Rejection is 2.
    '''
    #Primero definamos los riesgos de clasificar en 0, en 1 y el riesgo de rechazo
    risk0 = f_1(w, L, x, 0)
    risk1 = f_1(w, L, x, 1)
    risk2 = lamb
    #Si el riesgo de rechazar es menor que elegir cualquiera de las 2 clases, rechazamos
    if(risk2 < risk0 and risk2 < risk1):
      return 2
    #Si no rechazamos, miramos si el riesgo de clasificar en 1 es menor que en 0, si es asi
    #clasificamos en 1
    elif risk1 < risk0:
      return 1
    #Si nada de esto se cumple, clasificamos en 0
    else:
      return 0

In [19]:
# TEST CELL
lst = [[x1, x2] for x1 in np.linspace(-2, 1.5, 4) for x2 in np.linspace(-2, 1.5, 4)]
X = np.array(lst)
W1 = np.array([0, 1, 2])
L1 = np.array([[0.01, 2],[0.9, -0.05]]) 

EPS = 0.0001

Lamb1 = 0.3
C2= np.array([ 0.,  0.,  0.,  2.,  0.,  0.,  2.,  1.,  0.,  0.,  2.,  1.,  0., 2.,  1.,  1.])
# TEST CASES:
# CASE 1: code
evaluate = lambda i: ((abs(C2[i] - f_3(W1, L1, Lamb1, X[i, :])) < EPS) )
results = [evaluate(i) for i in range(len(lst))]
results == [True] * len(lst)

True